In [1]:
%cd /mnt/storage/maxenis/coptic-machine-translation

/mnt/storage/maxenis/coptic-machine-translation


In [2]:
import pandas as pd
import datasets
import huggingface_model
from config_consts import *
import parse_data

In [2]:
df = pd.DataFrame({'norm_group': [], 'translation': []})

In [3]:
# Add row to df
df.loc[0] = ["ϯⲛⲁϣⲗⲏⲗ ϧⲉⲛ ϯⲉⲕⲕⲗⲏⲥⲓⲁ ⲛⲣⲁⲥϯ", "I’ll pray in the church tomorrow"]
df.loc[1] = ["ϥⲛⲁⲓ ⲛⲉⲙ ⲡⲉϥⲓⲱⲧ ⲉ ⲡⲉⲛⲏⲓ", "he will come with his father to our house"]
df.loc[2] = ["ⲧⲉⲛⲛⲁⲥⲱⲧⲉⲙ ⲉ ϯⲁⲛⲁⲫⲟⲣⲁ ⲛⲧⲉ ⲡⲉⲛⲓⲱⲧ ⲙⲟⲣⲕⲟⲥ", "we shall listen to the mass of our father Morkos"]
df.loc[3] = ["ⲛⲓⲁⲗⲱⲟⲩⲓ ⲥⲉⲛⲁⲁⲣⲉϩ ⲉ ⲛⲟⲩⲱϣ", "the children will study their lessons"]
df.loc[4] = ["ϯⲛⲁϭⲓ ⲛϩⲁⲛⲗⲟⲩⲕⲟⲩϫⲓ ⲉⲃⲟⲗ ϧⲉⲛ ⲡⲁⲥⲟⲛ", "I’ll take some money from my brother"]
df.loc[5] = ["ϯⲛⲁⲓ ⲁⲛ ⲛⲣⲁⲥϯ ⲁⲗⲗⲁ ⲧⲛⲁⲓ ⲛⲥⲁⲣⲁⲥϯ", "I shall not come tomorrow, but I shall come the day after tomorrow"]
df.loc[6] = ["ϯⲛⲁⲙⲟⲩⲕϩ ⲁⲙⲁϣⲱ ⲓⲥϫⲉ ⲕⲥⲱⲧⲉⲙ ⲁⲛ", "I’ll be very upset if you don't listen"]
df.loc[7] = ["ⲧⲉⲛⲛⲁϣⲱⲡ ⲛϩⲁⲛϩⲣⲏⲣⲓ ⲉⲑⲃⲉ ⲧⲉⲛⲙⲁⲩ", "we shall buy some flowers for our mother"]
df.loc[8] = ["ⲧⲉⲛⲁⲭⲱ ⲛⲟⲩϩⲙⲟⲩ ⲉϫⲉⲛ ϯϧⲣⲉ", "will you put salt on the food?"]
df.loc[9] = ["ⲕⲟⲩⲱϣ ⲉⲟⲩⲱⲙ ϯⲛⲟⲩ", "do you want to eat now?"]
df.loc[10] = ["ϯⲥⲱⲟⲩⲛ ⲁⲛ ⲓⲥϫⲉ ϥⲛⲁⲓ ⲓⲉ ⲙⲙⲟⲛ", "I do not know whether he will come or not"]

In [4]:
df.to_csv('../datasets/bohairic.csv', index=False)

In [15]:
df = pd.read_csv('datasets/bohairic.csv')

In [17]:
df = parse_data.greekify_columns(df, ["norm_group"])

In [19]:
df.to_csv('datasets/bohairic-greekified.csv', index=False)

In [20]:
data = datasets.load_dataset('csv', data_files='datasets/bohairic-greekified.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
data["train"]["norm_group_greekified"]

['tναsληλ kεν tεκκλησια νρασt',
 'fναι νεμ πεfιωτ ε πενηι',
 'τεννασωτεμ ε tαναφορα ντε πενιωτ μορκοσ',
 'νιαλωουι σενααρεh ε νουωs',
 'tναcι νhανλουκουjι εβολ kεν πασον',
 'tναι αν νρασt αλλα τναι νσαρασt',
 'tναμουκh αμαsω ισjε κσωτεμ αν',
 'τενναsωπ νhανhρηρι εθβε τενμαυ',
 'τεναχω νουhμου εjεν tkρε',
 'κουωs εουωμ tνου',
 'tσωουν αν ισjε fναι ιε μμον']

In [23]:
data = data.map(lambda x: {'translation': {"cop": x['norm_group_greekified'], "eng": x['translation']}})

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['norm_group', 'translation', 'norm_group_greekified'],
        num_rows: 11
    })
})

In [28]:
data['train']

Dataset({
    features: ['norm_group', 'translation', 'norm_group_greekified'],
    num_rows: 11
})

In [26]:
data.save_to_disk('../datasets/bohairic-greekified')

Saving the dataset (0/1 shards):   0%|          | 0/11 [00:00<?, ? examples/s]

In [12]:
model = huggingface_model.HuggingFaceTranslationModel.from_pretrained('models/hf/fifth_attempt-norm_group_greekified-finetuned-cop-eng')


In [30]:
data = parse_data.load_test_set("datasets/bohairic-greekified.csv", "cop", "eng", "norm_group_greekified", "translation")
model.translate_test_data(data, BEAM_GENERATION_CONFIG)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Computing translations from cop-eng.


  0%|          | 0/11 [00:00<?, ?it/s]/home/bizon/anaconda3/envs/coptic/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
 82%|████████▏ | 9/11 [00:02<00:00,  5.31it/s]Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will ta

Translations completed. Computing metrics...
BLEU = 1.54 15.5/3.8/0.4/0.2 (BP = 1.000 ratio = 1.527 hyp_len = 142 ref_len = 93)
chrF2 = 21.29
